In [1]:
from epo.tipdata.patstat import PatstatClient
patstat = PatstatClient()

This client instance is currently configured to use a test dataset with reduced number of publications (~10K).
Use PatstatClient(env='PROD') to use the complete PATSTAT dataset (>140M publications).
Use PatstatClient(env='TEST') to use the test dataset and avoid displaying this warning



In [2]:
db = patstat.orm()

In [ ]:
query="""SELECT
    EXTRACT(YEAR FROM t1.APPLN_FILING_DATE) AS filing_year,
    COUNT(DISTINCT t1.APPLN_ID) AS patent_count
FROM TLS201_APPLN AS t1
JOIN TLS209_APPLN_IPC AS t2
    ON t1.APPLN_ID = t2.APPLN_ID
WHERE
    t2.IPC_CLASS_SYMBOL LIKE 'H02S%'
    AND t1.APPLN_FILING_DATE >= DATE '2000-01-01'
GROUP BY
    EXTRACT(YEAR FROM t1.APPLN_FILING_DATE)
ORDER BY
    filing_year"""
query = """
SELECT
    EXTRACT(YEAR FROM a.appln_filing_date) AS filing_year,
    COUNT(DISTINCT a.appln_id) AS patent_count
FROM tls201_appln a
JOIN tls209_appln_ipc i
  ON a.appln_id = i.appln_id
WHERE
    i.ipc_class_symbol LIKE 'H02S%'
    AND a.appln_filing_date >= DATE '2000-01-01'
GROUP BY
    filing_year
ORDER BY
    filing_year
"""


In [ ]:
res = patstat.sql_query(query, use_legacy_sql=False)
res

[{'filing_year': 2000, 'patent_count': 5},
 {'filing_year': 2001, 'patent_count': 7},
 {'filing_year': 2002, 'patent_count': 5},
 {'filing_year': 2003, 'patent_count': 5},
 {'filing_year': 2004, 'patent_count': 6},
 {'filing_year': 2005, 'patent_count': 2},
 {'filing_year': 2006, 'patent_count': 10},
 {'filing_year': 2007, 'patent_count': 22},
 {'filing_year': 2008, 'patent_count': 34},
 {'filing_year': 2009, 'patent_count': 83},
 {'filing_year': 2010, 'patent_count': 123},
 {'filing_year': 2011, 'patent_count': 132},
 {'filing_year': 2012, 'patent_count': 98},
 {'filing_year': 2013, 'patent_count': 108},
 {'filing_year': 2014, 'patent_count': 245},
 {'filing_year': 2015, 'patent_count': 355},
 {'filing_year': 2016, 'patent_count': 492},
 {'filing_year': 2017, 'patent_count': 665},
 {'filing_year': 2018, 'patent_count': 815},
 {'filing_year': 2019, 'patent_count': 779},
 {'filing_year': 2020, 'patent_count': 968},
 {'filing_year': 2021, 'patent_count': 910},
 {'filing_year': 2022, 'pat

In [4]:
import pandas as pd
from sqlalchemy import text
 
query = """
SELECT
    EXTRACT(YEAR FROM a.appln_filing_date) AS filing_year,
    COUNT(DISTINCT a.appln_id) AS patent_count
FROM tls201_appln a
JOIN tls209_appln_ipc i
  ON a.appln_id = i.appln_id
WHERE
    i.ipc_class_symbol LIKE 'H02S%'
    AND a.appln_filing_date >= DATE '2000-01-01'
GROUP BY
    filing_year
ORDER BY
    filing_year
"""
 
result = db.execute(text(query))
df = pd.DataFrame(result.fetchall(), columns=result.keys())
 
df.head()

,filing_year,patent_count
0,2000,5
1,2001,7
2,2002,5
3,2003,5
4,2004,6


In [5]:
len(df)

25

In [ ]:
CEU_sqls =[{"query":"""SELECT
    EXTRACT(YEAR FROM t1.appln_filing_date) AS filing_year,
    COUNT(
        DISTINCT CASE
            WHEN t2.ipc_class_symbol LIKE 'H02S%' THEN t1.appln_id
        END
    ) AS h02s_count,
    COUNT(
        DISTINCT CASE
            WHEN t2.ipc_class_symbol LIKE 'H10F%' THEN t1.appln_id
        END
    ) AS h10f_count,
    COUNT(
        DISTINCT CASE
            WHEN t2.ipc_class_symbol LIKE 'H01L  31%' THEN t1.appln_id
        END
    ) AS h01l31_count
FROM tls201_appln AS t1
JOIN tls209_appln_ipc AS t2
    ON t1.appln_id = t2.appln_id
WHERE
    t1.appln_filing_date >= DATE '2000-01-01'
    AND (
        t2.ipc_class_symbol LIKE 'H02S%'
        OR t2.ipc_class_symbol LIKE 'H10F%'
        OR t2.ipc_class_symbol LIKE 'H01L  31%'
    )
GROUP BY
    EXTRACT(YEAR FROM t1.appln_filing_date)
ORDER BY
    filing_year;
""", "name":"comparison of PV installation mechanics vs tech in semiconductors","description":"this compares the patstat data classified under H02S which is the part of PV tech dealing with mechanics of installations vs the technology in semiconductors technology itself"},
          ]

q={"query":"""SELECT
    EXTRACT(YEAR FROM t1.appln_filing_date) AS filing_year,
    COUNT(
        DISTINCT CASE
            WHEN t2.ipc_class_symbol LIKE 'H01L  31%' THEN t1.appln_id
        END
    ) AS h01l31_count,
    COUNT(
        DISTINCT CASE
            WHEN t2.ipc_class_symbol LIKE 'H10K  30%' THEN t1.appln_id
        END
    ) AS h10k30_count
FROM tls201_appln AS t1
JOIN tls209_appln_ipc AS t2
    ON t1.appln_id = t2.appln_id
WHERE
    t1.appln_filing_date >= DATE '2000-01-01'
    AND (
        t2.ipc_class_symbol LIKE 'H01L  31%'
        OR t2.ipc_class_symbol LIKE 'H10K  30%'
    )
GROUP BY
    EXTRACT(YEAR FROM t1.appln_filing_date)
ORDER BY
    filing_year;
""", "name":"comparing pv based on semiconductors technology vs organic based materials","description":"This data compares the applciations of pv modules based on inorganic materials, i.e. clasical semiconductors materials, versus ornaic based materials"}

In [13]:
CEU_sqls.append(q)

In [14]:
query = CEU_sqls[1]['query']

result = db.execute(text(query))
df = pd.DataFrame(result.fetchall(), columns=result.keys())
 
df.head()

,filing_year,h01l31_count,h10k30_count
0,2000,28,0
1,2001,51,0
2,2002,39,0
3,2003,52,0
4,2004,51,0
